In [1]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz, process

c:\Users\HEFRY ANESTI\AppData\Local\Programs\Python\Python39\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [10]:
eight_year_ranges = [(y, y+4) for y in range(1968, 2022, 5)]
five_yearIncrements = range(1970, 2021, 5)

for year, five_year in zip(eight_year_ranges, five_yearIncrements):
    print(year, five_year)

(1968, 1972) 1970
(1973, 1977) 1975
(1978, 1982) 1980
(1983, 1987) 1985
(1988, 1992) 1990
(1993, 1997) 1995
(1998, 2002) 2000
(2003, 2007) 2005
(2008, 2012) 2010
(2013, 2017) 2015
(2018, 2022) 2020


In [2]:
fubc_1_to_9 = pd.read_csv("D:/kerja/asisten riset/code v/review/doi_10_5061_dryad_2rbnzs7qh__v20220822/FUBC_1_to_9_data.csv")
fubc_9_raw_data = pd.read_csv("D:/kerja/asisten riset/code v/review/doi_10_5061_dryad_2rbnzs7qh__v20220822/FUBC_9_raw_data.csv", delimiter=',', encoding='latin', low_memory=False)
ifa_regions = pd.read_csv("D:/kerja/asisten riset/code v/review/doi_10_5061_dryad_2rbnzs7qh__v20220822/IFA_regions.csv", delimiter=',', encoding='latin', low_memory=False)

### Ambil kolom N fertilizer application (kg N/ha) tiap negara tiap crop

In [3]:
fubc_1_to_9.head()

,Original_country_name_in_FUBC_report,Country,ISO3_code,Region_IFA,Year,FUBC_report_number,Year_FUBC_publication,Crop,Crop_area_k_ha,N_k_t,...,N_rate_kg_ha,P2O5_rate_kg_ha,K2O_rate_kg_ha,N_pc_fert,P2O5_pc_fert,K2O_pc_fert,Aver_N_rate_kg_ha,Aver_P2O5_rate_kg_ha,Aver_K2O_rate_kg_ha,Aver_N_P2O5_K2O_rate_kg_ha
0,ALGERIA,Algeria,DZA,Africa,1986,1,1992,Barley,1212.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ALGERIA,Algeria,DZA,Africa,1986,1,1992,carrot,17.0,1.7,...,100.0,108.0,108.0,100.0,100.0,100.0,NaN,NaN,NaN,NaN
2,ALGERIA,Algeria,DZA,Africa,1986,1,1992,citrus Fruits,45.0,11.3,...,250.0,80.0,100.0,100.0,100.0,100.0,NaN,NaN,NaN,NaN
3,ALGERIA,Algeria,DZA,Africa,1986,1,1992,Fallow,2900.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
4,ALGERIA,Algeria,DZA,Africa,1986,1,1992,Fruits (Other),34.0,3.4,...,100.0,80.0,100.0,100.0,100.0,100.0,NaN,NaN,NaN,NaN


In [56]:
fubc_1_to_9_copy = fubc_1_to_9[['Original_country_name_in_FUBC_report', 'Country', 'ISO3_code',
       'Region_IFA', 'Year', 'Year_FUBC_publication',
       'Crop','N_rate_kg_ha']]
fubc_1_to_9_new = fubc_1_to_9_copy.copy()
fubc_1_to_9_new.rename(columns={'Crop': 'Item'}, inplace=True)
fubc_1_to_9_new

,Original_country_name_in_FUBC_report,Country,ISO3_code,Region_IFA,Year,Year_FUBC_publication,Item,N_rate_kg_ha
0,ALGERIA,Algeria,DZA,Africa,1986,1992,Barley,NaN
1,ALGERIA,Algeria,DZA,Africa,1986,1992,carrot,100.0
2,ALGERIA,Algeria,DZA,Africa,1986,1992,citrus Fruits,250.0
3,ALGERIA,Algeria,DZA,Africa,1986,1992,Fallow,0.0
4,ALGERIA,Algeria,DZA,Africa,1986,1992,Fruits (Other),100.0
...,...,...,...,...,...,...,...,...
7524,Vietnam,Viet Nam,VNM,East Asia,2018,2022,Rice,NaN
7525,Vietnam,Viet Nam,VNM,East Asia,2018,2022,Roots/tubers,NaN
7526,Vietnam,Viet Nam,VNM,East Asia,2018,2022,Soybeans,NaN
7527,Vietnam,Viet Nam,VNM,East Asia,2018,2022,Sugar cane,NaN


In [9]:
crop_class = pd.read_excel("D:/kerja/asisten riset/vol/milkunC/achaidir/FAO/Crop Classification_latest.xlsx", engine="openpyxl", skiprows=1)
crop_class = crop_class.drop('Unnamed: 0', axis=1)
crop_class.rename(columns={'FAO Crops': 'Item'}, inplace=True)

In [27]:
crop_class.loc[:,'Item'] = crop_class['Item'].str.lower()
crop_class['Item'].unique()

array(['maize (corn)', 'oil palm fruit',
       'anise, badian, coriander, cumin, caraway, fennel and juniper berries, raw',
       'chillies and peppers, dry (capsicum spp., pimenta spp.), raw',
       'chillies and peppers, green (capsicum spp. and pimenta spp.)',
       'cinnamon and cinnamon-tree flowers, raw',
       'cloves (whole stems), raw', 'cocoa beans', 'coffee, green',
       'ginger, raw', 'green garlic', 'hop cones', 'maté leaves',
       'natural rubber in primary forms', 'nutmeg, mace, cardamoms, raw',
       'other stimulant, spice and aromatic crops, n.e.c.',
       'pepper (piper spp.), raw', 'peppermint, spearmint', 'persimmons',
       'pyrethrum, dried flowers', 'tea leaves', 'unmanufactured tobacco',
       'vanilla, raw', 'barley', 'buckwheat', 'canary seed',
       'cereals n.e.c.', 'mixed grain', 'oats', 'rye', 'triticale',
       'abaca, manila hemp, raw', 'agave fibres, raw, n.e.c.',
       'flax, processed but not spun', 'jute, raw or retted',
       'kena

In [57]:
fubc_1_to_9_new.loc[:,'Item'] = fubc_1_to_9_new['Item'].str.replace('\r\r\r\n', '')
fubc_1_to_9_new.loc[:,'Item'] = fubc_1_to_9_new['Item'].str.replace('/', '')
fubc_1_to_9_new.loc[:,'Item'] = fubc_1_to_9_new['Item'].str.strip()
fubc_1_to_9_new.loc[:,'Item'] = fubc_1_to_9_new['Item'].str.lower()
fubc_1_to_9_new['Item'].unique()

array(['barley', 'carrot', 'citrus fruits', 'fallow', 'fruits (other)',
       'garlic', 'grape', 'olive', 'onion', 'other', 'pasture &  fodder',
       'pasture & fodder (other)', 'potato', 'pulses', 'rough',
       'vegetables (other)', 'wheat', 'wheat (durum)', 'banana', 'maize',
       'vegetables', 'cereals', 'cotton', 'oil crops', 'sugar cane',
       'tobacco', 'cereals (other)', 'fodder crop',
       'grassland (intensive)', 'grassland (moderate)', 'oat', 'rapeseed',
       'rye', 'sugar beet', 'sunflower', 'jute', 'rice (aman)',
       'rice (aus)', 'rice (boro)', 'spices', 'grasslands & pastures',
       'maize (green, silage)', 'cassava', 'groundnut', 'millet',
       'oil-palm', 'sorghum', 'peach', 'rice', 'tomato', 'castor bean',
       'cocoa', 'coconut', 'coffee', 'forestry', 'orange',
       'pasture & fodder', 'pepper and chilli', 'pineapple',
       'rootstubers (other)', 'soybean', 'fruits', 'cabbage',
       'canteloupeothers', 'cucumber', 'eggplant', 'flax', 'linse

In [83]:
from fuzzywuzzy import fuzz, process

# Definisikan data yang ingin diganti
data_fubc_1_to_9_new = fubc_1_to_9_new['Item'].to_list()

# Definisikan data yang ingin digunakan sebagai pengganti
data_crop_class = crop_class['Item'].to_list()

# Konversi data ke tipe string
data_ganti = [str(x) for x in data_fubc_1_to_9_new]
data_pengganti = [str(x) for x in data_crop_class]

In [86]:
hasil = []
for x in data_ganti:
    hasil.append(process.extract(x, data_pengganti, limit=1))
    
# Tampilkan hasil
hasil

In [ ]:
hasil_df = pd.DataFrame(hasil, columns=['Item', 'Score'])
fubc_1_to_9_new.loc[:, 'Item'] = hasil_df['Item']

In [ ]:
fubc_1_to_9_merge = fubc_1_to_9_new.merge(crop_class, on='Item')